In [1]:
import torch
from torch import Tensor                  
import torch.nn as nn                     

import matplotlib.pyplot as plt

import numpy as np
import time

torch.set_default_tensor_type(torch.DoubleTensor)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from tqdm import tqdm
print(device)

cuda


In [2]:
# for real roots
class NNR(nn.Module):
    
    def __init__(self,layers):
        super().__init__() 
              
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]).to(device)
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            nn.init.zeros_(self.linears[i].bias.data)
            
    def forward(self,x):
            if torch.is_tensor(x) !=True:
                x= torch.from_numpy(x).to(device)
            sigma = x.type(torch.DoubleTensor).to(device)
            for i in range(len(layers)-2):
                z = self.linears[i](sigma)
                sigma = self.activation(z)
            sigma = self.linears[-1](sigma)
            return sigma
    
    #Modify the loss function as per the problem
    def loss_func(self, x_train,n):                     
        g = x_train.clone()
                        
        g.requires_grad = True
        
        u = self.forward(g)
                
        loss_f = self.loss_function(eqn(u,x_train,n),torch.tensor(0).type(torch.DoubleTensor).to(device))   
        return loss_f
                                           
    def closure(self,steps,n,eps=1e-8,lr=1e-2,show=True):
            start = time.time()
            optimizer = torch.optim.Adam(self.parameters(),lr=lr)
            for i in tqdm(range(steps)):
                loss = self.loss_func(x_train,n)
                self.mse = loss
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #Learning rate scheduling. It performs better using this even for Adam.
                if i%(steps/4)==0:
                    lr=lr/5
                    optimizer = torch.optim.Adam(self.parameters(),lr=lr)
                    if show==True:
                        with torch.no_grad():
                            print('Iter: ',i,'Loss: ',loss.detach().cpu().numpy(),' lr: ',lr)
                if self.mse<=eps:
                    print('Converged !')
                    break
            print('MSE Loss: ',float(self.mse.detach().cpu()))
            print('total time: ',time.time()-start)

In [3]:
# algorithm for dividing polynomials
def poly_div(dividend, divisor):
    n = len(dividend)
    m = len(divisor)
    quotient = [0] * (n - m + 1)
    remainder = dividend.copy()
    for i in range(n - m, -1, -1):
        quotient[i] = remainder[m + i - 1] / divisor[m - 1]
        for j in range(m):
            remainder[i + j] -= quotient[i] * divisor[j]
    return quotient, remainder[:m - 1]

In [4]:
def eqn(u,x_train,n):
    pol=torch.pow(u,n)
    for i in range(n):
        pol=pol+x_train[:,[i]]*torch.pow(u,n-i-1)
    return pol

In [5]:
# 3rd order
b = np.linspace(-2,2,50)
c = np.linspace(-2,2,50)
d = np.linspace(-2,2,50)

B,C,D = np.meshgrid(b,c,d)
x_train = torch.from_numpy(np.hstack(( B.flatten()[:,None],C.flatten()[:,None],
                                      D.flatten()[:,None]))).to(device)
x_train.shape

torch.Size([125000, 3])

In [6]:
steps=10000
layers = np.array([3,20,20,20,20,20,20,20,20,1])
Root_NN3 = NNR(layers)
Root_NN3.to(device)
Root_NN3.closure(steps=steps,n=3,show=True)

  0%|                                                 | 0/10000 [00:00<?, ?it/s]/home/djc/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([125000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|                                         | 9/10000 [00:00<04:44, 35.08it/s]

Iter:  0 Loss:  1.4697214930321882  lr:  0.002


 25%|█████████▌                            | 2508/10000 [00:34<01:42, 73.44it/s]

Iter:  2500 Loss:  0.00259279587785947  lr:  0.0004


 50%|███████████████████                   | 5009/10000 [01:09<01:08, 73.13it/s]

Iter:  5000 Loss:  0.0009064353023678338  lr:  8e-05


 75%|████████████████████████████▌         | 7513/10000 [01:43<00:33, 73.40it/s]

Iter:  7500 Loss:  0.0006891116390388397  lr:  1.6000000000000003e-05


100%|█████████████████████████████████████| 10000/10000 [02:16<00:00, 73.09it/s]

MSE Loss:  0.0005921365839955234
total time:  136.81823348999023


In [7]:
torch.save(Root_NN3,'models/Cubic.pt')
del Root_NN3

In [6]:
# for complex roots
class NNCR(nn.Module):
    
    def __init__(self,layers):
        super().__init__() 
              
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]).to(device)
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            nn.init.zeros_(self.linears[i].bias.data)
            
    def forward(self,x):
            if torch.is_tensor(x) !=True:
                x= torch.from_numpy(x).to(device)
            sigma = x.type(torch.DoubleTensor).to(device)
            for i in range(len(layers)-2):
                z = self.linears[i](sigma)
                sigma = self.activation(z)
            sigma = self.linears[-1](sigma)
            return sigma

    
    #Modify the loss function as per the problem
    def loss_func(self, x_train1):                     
        g = x_train1.clone()
                        
        g.requires_grad = True
        
        u = self.forward(g)
        real,imag = com_eqn(u,x_train1)        
        return (real**2).mean()+(imag**2).mean()

                                           
    def closure(self,steps,eps=1e-8,lr=1e-1,show=True):
            start = time.time()
            optimizer = torch.optim.Adam(self.parameters(),lr=lr)
            for i in tqdm(range(steps)):
                loss = self.loss_func(x_train1)
                self.mse = loss
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #Learning rate scheduling. It performs better using this even for Adam.
                if i%(steps/4)==0:
                    lr=lr/5
                    optimizer = torch.optim.Adam(self.parameters(),lr=lr)
                    if show==True:
                        with torch.no_grad():
                            print('Iter: ',i,'Loss: ',loss.detach().cpu().numpy(),' lr: ',lr)
                if self.mse<=eps:
                    print('Converged !')
                    break
            print('MSE Loss: ',float(loss.detach().cpu().numpy()))
            print('total time: ',time.time()-start)

In [7]:
def com_eqn(u,x_train1):
    x = u[:,[0]]
    y= u[:,[1]]
    b=x_train1[:,[0]]
    c=x_train1[:,[1]]
    return (x**2-y**2+b*x+c),(2*x*y+b*y)

In [10]:
# 2nd order
b = np.linspace(-5,5,500)
c = np.linspace(-5,5,500)
B,C = np.meshgrid(b,c)
x_train1 = torch.from_numpy(np.hstack(( B.flatten()[:,None],C.flatten()[:,None]))).to(device)

steps=10000
layers = np.array([2,50,50,50,50,50,50,2])
Root_NN2 = NNCR(layers)
Root_NN2.to(device)
Root_NN2.closure(steps=steps,show=True)

  0%|                                         | 4/10000 [00:00<05:02, 33.01it/s]

Iter:  0 Loss:  8.840611156438738  lr:  0.02


 25%|█████████▌                            | 2504/10000 [01:14<03:46, 33.04it/s]

Iter:  2500 Loss:  0.010104551923155975  lr:  0.004


 50%|███████████████████                   | 5004/10000 [02:29<02:29, 33.45it/s]

Iter:  5000 Loss:  0.001461997881196653  lr:  0.0008


 75%|████████████████████████████▌         | 7504/10000 [03:43<01:14, 33.58it/s]

Iter:  7500 Loss:  0.0008577223383164048  lr:  0.00016


100%|█████████████████████████████████████| 10000/10000 [04:57<00:00, 33.62it/s]

MSE Loss:  0.0003930701666850679
total time:  297.43820357322693


In [11]:
torch.save(Root_NN2,'models/Complex_quad.pt')

In [12]:
# 4rd order
b = np.linspace(0,2,25)
c = np.linspace(0,2,25)
d = np.linspace(0,2,25)
e = np.linspace(-2,0,25)
B,C,D,E = np.meshgrid(b,c,d,e)
x_train = torch.from_numpy(np.hstack(( B.flatten()[:,None],C.flatten()[:,None],
                                      D.flatten()[:,None],E.flatten()[:,None]))).to(device)

steps=10000
layers = np.array([4,20,20,20,20,20,20,20,20,1])
Root_NN4 = NNR(layers)
Root_NN4.to(device)
Root_NN4.closure(steps=steps,n=4,show=True)

  0%|                                                 | 0/10000 [00:00<?, ?it/s]/home/djc/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([390625, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|                                         | 5/10000 [00:00<07:26, 22.37it/s]

Iter:  0 Loss:  2.4381411504344594  lr:  0.002


 25%|█████████▌                            | 2504/10000 [01:40<05:02, 24.80it/s]

Iter:  2500 Loss:  0.000105973121632195  lr:  0.0004


 50%|███████████████████                   | 5003/10000 [03:21<03:21, 24.77it/s]

Iter:  5000 Loss:  3.646943897663825e-05  lr:  8e-05


 75%|████████████████████████████▌         | 7505/10000 [05:02<01:40, 24.81it/s]

Iter:  7500 Loss:  2.0782449272964285e-05  lr:  1.6000000000000003e-05


100%|█████████████████████████████████████| 10000/10000 [06:43<00:00, 24.80it/s]

MSE Loss:  1.5083995924848818e-05
total time:  403.2259407043457


In [13]:
torch.save(Root_NN4,'models/Quartic.pt')

# Tests for all roots

In [51]:
#Cubic1
test = np.array([1,1.5,1,-1.5])
layers = np.array([3,20,20,20,20,20,20,20,20,1])
NN3 = torch.load('models/Cubic.pt')
root1 = NN3(test[1:]).detach().cpu().numpy()[0]
poly2,rem = poly_div(test,[1,-1*root1])
layers = np.array([2,50,50,50,50,50,50,2])
NN2 = torch.load('models/Complex_quad.pt')
root2 = NN2(np.array(poly2[1:])).detach().cpu().numpy()
root3 = [root2[0],-root2[1]]
print('roots are %1.5f, %1.5f+%1.5fi, %1.5f+%1.5fi'%(root1,root2[0],root2[1],root3[0],root3[1]))

roots are 0.63578, -1.06777+1.09805i, -1.06777+-1.09805i


In [9]:
#Cubic2
test = np.array([1,0.5,1,-0.5])

layers = np.array([3,20,20,20,20,20,20,20,20,1])
NN3 = torch.load('models/Cubic.pt')
root1 = NN3(test[1:]).detach().cpu().numpy()[0]
poly2,rem = poly_div(test,[1,-1*root1])
layers = np.array([2,50,50,50,50,50,50,2])
NN2 = torch.load('models/Complex_quad.pt')
root2 = NN2(np.array(poly2[1:])).detach().cpu().numpy()
root3 = [root2[0],-root2[1]]
print('roots are %1.5f, %1.5f+%1.5fi, %1.5f+%1.5fi'%(root1,root2[0],root2[1],root3[0],root3[1]))

roots are 0.37606, -0.43420+1.06800i, -0.43420+-1.06800i


In [49]:
#Quartic
test4 =  [1,1,1,1,-1]
layers = np.array([4,20,20,20,20,20,20,20,20,1])
NN4 = torch.load('models/Quartic.pt')
root1 = NN4(np.array(test4[1:])).detach().cpu().numpy()[0]
poly3,rem = poly_div(test4,[1,-root1])

layers = np.array([3,20,20,20,20,20,20,20,20,1])
NN3 = torch.load('models/Cubic.pt')
root2 = NN3(np.array(poly3[1:])).detach().cpu().numpy()[0]
poly2,rem = poly_div(poly3,[1,-root2])

layers = np.array([2,50,50,50,50,50,50,2])
NN2 = torch.load('models/Complex_quad.pt')
root3 = NN2(np.array(poly2[1:])).detach().cpu().numpy()
root4 = [root3[0],-root3[1]]
print('roots are %1.5f, %1.5f, %1.5f+%1.5fi, %1.5f+%1.5fi'%(root1,root2,root4[0],root4[1],root3[0],root3[1]))

roots are 0.51839, -1.29037, -0.10957+1.21448i, -0.10957+-1.21448i
